## Income Prediction Project

### Summary

The US Census reports a vast dataset of demographic information about people living in the USA. A subset of this data which has been munged to a certain degree can be found [here](https://archive.ics.uci.edu/ml/datasets/Census+Income) with 15 features, including income information.

In this analysis, I built two models using random forest classifier (RFC) and losgistic regression (LG). RFC achieved 85.1% and 84.9% accuracy on the training data and the test data seperately, while LG got 77.2% and 77.0% for training and test data.

### Analysis

#### Import modules

In [60]:
import pandas as pd
import numpy as np
from pprint import pprint
from time import time

from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedShuffleSplit

#### Read in data and feature conversion

In [61]:
colnames = ['age', 'workclass', 'fnlwgt', 'education', \
            'education_num', 'marital_status', 'occupation', \
            'relationship', 'race', 'sex', 'capital_gain', \
            'capital_loss', 'hours_per_week', 'native_country',\
            'income']
income_data = pd.read_csv("adult.data.csv", 
                          names = colnames)
income_data.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
income            object
dtype: object

First thing, we need to check if there's any duplicated information in the data and then remove them accordingly.

In [62]:
income_data = income_data.drop_duplicates()

Then we need to convert the categorical variables. 

In [63]:
from sklearn.preprocessing import LabelEncoder
le_workclass = LabelEncoder()
le_education = LabelEncoder()
le_marital_status = LabelEncoder()
le_occupation = LabelEncoder()
le_relationship = LabelEncoder()
le_race = LabelEncoder()
le_sex = LabelEncoder()
le_native_country = LabelEncoder()
le_income = LabelEncoder()

### To convert into numbers
income_data.workclass = le_workclass.fit_transform(income_data.workclass)
income_data.education = le_education.fit_transform(income_data.education)
income_data.marital_status = le_marital_status.fit_transform(income_data.marital_status)
income_data.occupation = le_occupation.fit_transform(income_data.occupation)
income_data.relationship = le_relationship.fit_transform(income_data.relationship)
income_data.race = le_race.fit_transform(income_data.race)
income_data.sex = le_sex.fit_transform(income_data.sex)
income_data.native_country = le_native_country.fit_transform(income_data.native_country)
income_data.income = le_income.fit_transform(income_data.income)

### To convert back for easy explaination
# income_data.sex = le_sex.inverse_transform(income_data.sex)

In [64]:
income_data[(income_data.capital_gain > 0) & (income_data.capital_loss > 0)]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income


For the query above, we see there's no records with "capital_gain" and "capital_loss" greater than zero at the same time. It verified my assumption that the total amount of capital is shown in two seperated features -- when the number is positive it's shown in "capital_gain" and if it's negative then the absolute number is shown in "capital_loss". Therefore, I created a variable called "capital_tot" as following to summarise the information.

In [65]:
income_data["capital_tot"] = income_data.capital_gain - income_data.capital_loss

#### Feature Selection

First, I want to narrow the candidates of features to choose from. I'd keep only one feature, the more informative one, from each highly correlated group of features.  

In [66]:
col_sml = ['age', 'workclass', 'fnlwgt', \
 'education', 'marital_status', 'occupation',\
 'relationship', 'race', 'sex', 'capital_tot', \
 'hours_per_week', 'native_country']

The candidates are all the "col_sml" list shown above. "education_num" was removed since it's highly correlated with "education" and "capital_gain" and "capital_loss" were removed and only the newly created feature of "capital_tot" was kept.

In [67]:
features_to_choose = income_data[col_sml].values

In [68]:
def select_k_best_features(col_names, features, labels, k):
    """
    For Census Income dataset, select k best features based on SelectKBest from 
    sklearn.feature_selection

    Input:
    dataset: data in dictionary format 
    k: the number of features to keep

    Return:
    the list of length of k: k best features 
    """    
    k_best = SelectKBest(k=k)
    k_best.fit(features, labels)
    impt_unsorted = zip(col_names, k_best.scores_)
    impt_sorted = list(sorted(impt_unsorted, key=lambda x: x[1], reverse=True))
    k_best_features = [elem[0] for elem in impt_sorted][:k]
    print k, "best features:"
    print k_best_features
    return k_best_features

Then use `SelectKBest` to get the k best features in term of their scores.

In [69]:
labels, features = income_data['income'].values, income_data[col_sml].values
K_features = income_data[select_k_best_features(col_sml, features, labels, 9)].values

9 best features:
['relationship', 'age', 'hours_per_week', 'sex', 'capital_tot', 'marital_status', 'education', 'occupation', 'race']


#### All the features and their scores

In [70]:
k_best = SelectKBest(k=10)
k_best.fit(features, labels)
impt_unsorted = zip(col_sml, k_best.scores_)
impt_unsorted

[('age', 1885.3164412192941),
 ('workclass', 87.076858983064071),
 ('fnlwgt', 2.9380014706440551),
 ('education', 206.23794343878359),
 ('marital_status', 1344.3447614104623),
 ('occupation', 186.11728830088055),
 ('relationship', 2186.594101437629),
 ('race', 168.81906294077584),
 ('sex', 1591.7637612837962),
 ('capital_tot', 1568.1679650910089),
 ('hours_per_week', 1811.5338655278399),
 ('native_country', 7.9381403319498256)]

I decided to keep nine features for future model(s) and discarded the three features ('workclass', 'fnlwgt', 'native_country') with score less than 100

#### Build Models

In [9]:
PERF_FORMAT_STRING = "\
\tAccuracy: {:>0.{display_precision}f}\tPrecision: {:>0.{display_precision}f}\t\
Recall: {:>0.{display_precision}f}\tF1: {:>0.{display_precision}f}\tF2: {:>0.{display_precision}f}"
RESULTS_FORMAT_STRING = "\tTotal predictions: {:4d}\tTrue positives: {:4d}\tFalse positives: {:4d}\tFalse negatives: {:4d}\tTrue negatives: {:4d}"

def test_classifier(clf, labels, features, folds = 100):
    """
    Use cross validation to evaluate the performance of models
    """
    cv = StratifiedShuffleSplit(labels, folds, random_state = 42)
    true_negatives = 0
    false_negatives = 0
    true_positives = 0
    false_positives = 0
    for train_idx, test_idx in cv: 
        features_train = []
        features_test  = []
        labels_train   = []
        labels_test    = []
        for ii in train_idx:
            features_train.append( features[ii] )
            labels_train.append( labels[ii] )
        for jj in test_idx:
            features_test.append( features[jj] )
            labels_test.append( labels[jj] )
        
        ### fit the classifier using training set, and test on test set
        clf.fit(features_train, labels_train)
        predictions = clf.predict(features_test)
        for prediction, truth in zip(predictions, labels_test):
            if prediction == 0 and truth == 0:
                true_negatives += 1
            elif prediction == 0 and truth == 1:
                false_negatives += 1
            elif prediction == 1 and truth == 0:
                false_positives += 1
            else:
                true_positives += 1
    try:
        total_predictions = true_negatives + false_negatives + false_positives + true_positives
        accuracy = 1.0*(true_positives + true_negatives)/total_predictions
        precision = 1.0*true_positives/(true_positives+false_positives)
        recall = 1.0*true_positives/(true_positives+false_negatives)
        f1 = 2.0 * true_positives/(2*true_positives + false_positives+false_negatives)
        f2 = (1+2.0*2.0) * precision*recall/(4*precision + recall)
        print clf
        print PERF_FORMAT_STRING.format(accuracy, precision, recall, f1, f2, display_precision = 5)
        print RESULTS_FORMAT_STRING.format(total_predictions, true_positives, false_positives, false_negatives, true_negatives)
        print ""
    except:
        print "Got a divide by zero when trying out:", clf


In [25]:
def best_parameter_from_search(pipeline, parameters, score_func, labels, features, kf = 10):
    """ 
    print out the optimal parameters of pipeline classifier from grid search based on 
    score function of choice
    
    Input:
    pipeline: classifier in pipeline form
    parameters: the parameters to be grid searched
    score_func: Scorer function used on the held out data to choose the best parameters for the model
    dataset: data in dictionary format
    features_list: the list of feature after feature selection
    kf: kf-fold of cross validation for estimation
    """
    ### Stratified ShuffleSplit cross validation iterator of the training set
    cv_sss = StratifiedShuffleSplit(labels, n_iter=kf, test_size=0.2, random_state=0)

    clf = GridSearchCV(pipeline, parameters, scoring=score_func, cv=cv_sss, n_jobs=-1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    clf.fit(features, labels)
    print "done in %0.3fs" % (time() - t0)
    print
    print("Best score: %0.3f" % clf.best_score_)
    print("Best parameters set:")
    best_parameters = clf.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    return clf

**Random Forest Classifier**

In [24]:
pipeline_rfc = Pipeline(steps=[
    ('classifier', RandomForestClassifier(random_state = 42))  
])

parameters_rfc = {
    'classifier__max_features': ('sqrt', 1),
    'classifier__max_depth': np.arange(3, 8),
    'classifier__n_estimators' : (10, 20)
}

In [31]:
best_parameter_from_search(pipeline_rfc, parameters_rfc, 'f1', labels, K_features)

Performing grid search...
('pipeline:', ['classifier'])
parameters:
{'classifier__max_depth': array([3, 4, 5, 6, 7]),
 'classifier__max_features': ('sqrt', 1),
 'classifier__n_estimators': (10, 20)}
Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  50 jobs       | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 194 out of 200 | elapsed:   15.5s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   16.0s finished


done in 16.419s

Best score: 0.628
Best parameters set:
	classifier__max_depth: 7
	classifier__max_features: 'sqrt'
	classifier__n_estimators: 20


GridSearchCV(cv=StratifiedShuffleSplit(labels=[0 0 ..., 0 1], n_iter=10, test_size=0.2, random_state=0),
       estimator=Pipeline(steps=[('classifier', RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0))]),
       fit_params={}, iid=True, loss_func=None, n_jobs=-1,
       param_grid={'classifier__max_features': ('sqrt', 1), 'classifier__n_estimators': (10, 20), 'classifier__max_depth': array([3, 4, 5, 6, 7])},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring='f1',
       verbose=1)

In [58]:
clf_rfc = RandomForestClassifier(max_depth = 7, 
                             max_features = 'sqrt', 
                             n_estimators = 20, 
                             random_state = 42)
test_classifier(clf_rfc, labels, K_features)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=7, max_features='sqrt',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=42, verbose=0)
	Accuracy: 0.85140	Precision: 0.79364	Recall: 0.51788	F1: 0.62677	F2: 0.55656
	Total predictions: 325400	True positives: 40602	False positives: 10557	False negatives: 37798	True negatives: 236443



**Logistic Regression**

In [35]:
pipeline_lrg = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(tol = 0.001, random_state = 42))
])

parameters_lrg = {
    'classifier__penalty': ('l1', 'l2'),
    'classifier__C': 10.0 ** np.arange(-10, -3)
    }

### Grid search for the optimal parameters
best_parameter_from_search(pipeline_lrg, parameters_lrg, 'f1', labels, K_features)

Performing grid search...
('pipeline:', ['scaler', 'classifier'])
parameters:
{'classifier__C': array([  1.00000000e-10,   1.00000000e-09,   1.00000000e-08,
         1.00000000e-07,   1.00000000e-06,   1.00000000e-05,
         1.00000000e-04]),
 'classifier__penalty': ('l1', 'l2')}
Fitting 10 folds for each of 14 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  50 jobs       | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 134 out of 140 | elapsed:    4.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:    4.8s finished
/Users/apple/anaconda/lib/python2.7/site-packages/sklearn/metrics/metrics.py:1771: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/apple/anaconda/lib/python2.7/site-packages/sklearn/metrics/metrics.py:1771: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/apple/anaconda/lib/python2.7/site-packages/sklearn/metrics/metrics.py:1771: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/apple/anaconda/lib/python2.7/site-pa

done in 4.986s

Best score: 0.520
Best parameters set:
	classifier__C: 9.9999999999999995e-08
	classifier__penalty: 'l2'


In [36]:
clf_lrg = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(tol = 0.001, C = 10**-8, penalty = 'l2', 
                                          random_state = 42))
])
test_classifier(clf_lrg, labels, K_features)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('classifier', LogisticRegression(C=1e-08, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=42, tol=0.001))])
	Accuracy: 0.77170	Precision: 0.52665	Recall: 0.51806	F1: 0.52232	F2: 0.51976
	Total predictions: 325400	True positives: 40616	False positives: 36505	False negatives: 37784	True negatives: 210495



#### Testing and Prediction

In [54]:
### Read id data
income_data_test = pd.read_csv("adult.test.csv", names = colnames,
                              skiprows = 1)

### Convert the categorical variables
income_data_test.workclass = le_workclass.fit_transform(income_data_test.workclass)
income_data_test.education = le_education.fit_transform(income_data_test.education)
income_data_test.marital_status = le_marital_status.fit_transform(income_data_test.marital_status)
income_data_test.occupation = le_occupation.fit_transform(income_data_test.occupation)
income_data_test.relationship = le_relationship.fit_transform(income_data_test.relationship)
income_data_test.race = le_race.fit_transform(income_data_test.race)
income_data_test.sex = le_sex.fit_transform(income_data_test.sex)
income_data_test.native_country = le_native_country.fit_transform(income_data_test.native_country)
income_data_test.income = le_income.fit_transform(income_data_test.income)

### Add new feature
income_data_test["capital_tot"] = income_data_test.capital_gain - income_data_test.capital_loss

In [55]:
labels_test, features_test = income_data_test['income'].values, income_data_test[col_sml].values
K_features_test = income_data_test[select_k_best_features(col_sml, features_test, labels_test, 9)].values

9 best features:
['relationship', 'hours_per_week', 'age', 'capital_tot', 'sex', 'marital_status', 'education', 'occupation', 'race']


**Prediction on Random Forest Classifier**

In [59]:
test_classifier(clf_rfc, labels_test, K_features_test)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=7, max_features='sqrt',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=42, verbose=0)
	Accuracy: 0.84900	Precision: 0.81129	Recall: 0.47055	F1: 0.59563	F2: 0.51370
	Total predictions: 162900	True positives: 18116	False positives: 4214	False negatives: 20384	True negatives: 120186



**Prediction on Logistic Regression**

In [56]:
test_classifier(clf_lrg, labels_test, K_features_test)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('classifier', LogisticRegression(C=1e-08, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=42, tol=0.001))])
	Accuracy: 0.77023	Precision: 0.51451	Recall: 0.49283	F1: 0.50344	F2: 0.49702
	Total predictions: 162900	True positives: 18974	False positives: 17904	False negatives: 19526	True negatives: 106496



###Conclusion and reflection

Overall, Random Forest Classifier overperforms Logistic Regression on prediction of income type. Also, we see the "accuracy", "recall", "precision" scores of each model were really consistent for both the training set and the test set, which means the training dataset is really representitave and our model were neither overfitting nor underfitting.  

What to improve next: 
1. Due to the time limit, I didn't investigate the outliers at the beginning. However, this step is really important in terms of parameter tuning.
2. Also, doing the correlation analysis first may also be a good idea since many algorithms, epecially Naive Bayes depends on the assumption that all the features are independent. Although I removed some features before carrying out `SelectKBest` intuitively to avoid the problem may caused by multicolinearity.
3. It would be interesting to try other algorithms as well, such as Naive Bayes and Support Vector Classifier etc. and compare the preformance of different models.